In [1]:
import pandas as pd
import networkx as nx
import altair as alt
import nx_altair as nxa
import ast
from vega_datasets import data

In [2]:
df = pd.read_excel('CountryEconomics.xlsx')

# GDP is in billions, Population is full count
df['GDP_full'] = df['GDP'] * 1_000_000_000    # convert billions → full units
df['GDP_per_capita'] = df['GDP_full'] / df['Population']

# Test

In [3]:
gdp_bar_chart = alt.Chart(df).mark_bar().encode(
    x='Country',
    y='GDP'
)
gdp_bar_chart.show()

alt.Chart(...)

# Map

In [4]:
world_map = alt.topo_feature(data.world_110m.url, 'countries')

# Calculate population density
df['Population_Density'] = df['Population'] / df['Area']

# Filter to only include countries with borders (no islands)
if not isinstance(df['Borders'].iloc[0], list):
    df['Borders'] = df['Borders'].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) else ([] if pd.isna(x) else x)
    )

df_with_borders = df[df['Borders'].apply(lambda x: len(x) > 0)].copy()

country_selection = alt.selection_point(
    fields=['Country'],
    empty=False,
    value=[{'Country': 'Hungary'}]
)

population_density_map = alt.Chart(world_map).mark_geoshape().encode(
    color=alt.condition(
        country_selection,
        alt.value('red'),
        alt.Color('Population_Density:Q', 
                  scale=alt.Scale(type='log'), 
                  legend=alt.Legend(orient='left', 
                                   title='Population Density',
                                   titleFontSize=14,
                                   labelFontSize=12))
    ),
    tooltip=['Country:N', 'Population_Density:Q']
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(df_with_borders, 'ID', ['Country', 'Population_Density'])
).project('mercator').properties(
    width=825,
    height=450,
    title=alt.TitleParams('People per square kilometre, Click to select (No Islands included)', fontSize=16)
).add_params(country_selection).interactive()

population_density_map


alt.Chart(...)

# Network

In [5]:
# Parse Borders column to Python lists (only if not already parsed)
if not isinstance(df['Borders'].iloc[0], list):
    df['Borders'] = df['Borders'].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) else ([] if pd.isna(x) else x)
    )

# Filter to only include countries with borders (exclude islands)
df_with_borders = df[df['Borders'].apply(lambda x: len(x) > 0)].copy()

# Create a set of valid 3-letter abbreviations in the dataset (only countries with borders)
valid_country_codes = set(df_with_borders['Abbreviation_3'].dropna().tolist())

# Build the graph using 3-letter abbreviations as node IDs
border_graph = nx.Graph()
for idx, row in df_with_borders.iterrows():
    country_code = row['Abbreviation_3']
    if pd.notna(country_code):  # Only add if we have a valid 3-letter code
        country_name = row['Country']
        # Add node with abbreviation as ID and country name as attribute
        border_graph.add_node(country_code, Country=country_name)
        # Add edges only for neighbors that exist in our dataset
        for neighbor_code in row['Borders']:
            if neighbor_code in valid_country_codes:
                border_graph.add_edge(country_code, neighbor_code)

# Visualize the network graph
node_positions = nx.spring_layout(border_graph, k=0.5,  seed=42)

# Convert NetworkX graph to DataFrames for Altair
# Create expanded edges DataFrame with all neighbor relationships
edges_with_neighbors = []
for edge in border_graph.edges():
    source, target = edge
    source_pos = node_positions[source]
    target_pos = node_positions[target]
    source_country = border_graph.nodes[source].get('Country', source)
    target_country = border_graph.nodes[target].get('Country', target)
    
    # Add edge twice - once for each direction
    edges_with_neighbors.append({
        'source_x': source_pos[0],
        'source_y': source_pos[1],
        'target_x': target_pos[0],
        'target_y': target_pos[1],
        'Country': source_country,
    })
    edges_with_neighbors.append({
        'source_x': source_pos[0],
        'source_y': source_pos[1],
        'target_x': target_pos[0],
        'target_y': target_pos[1],
        'Country': target_country,
    })

edges_df = pd.DataFrame(edges_with_neighbors)

# Create nodes DataFrame
nodes_data = []
for node in border_graph.nodes():
    country_name = border_graph.nodes[node].get('Country', node)
    pos = node_positions[node]
    neighbors = [border_graph.nodes[n].get('Country', n) for n in border_graph.neighbors(node)]
    nodes_data.append({
        'Country': country_name,
        'x': pos[0],
        'y': pos[1],
        'neighbors': neighbors
    })
nodes_df = pd.DataFrame(nodes_data)

# Add connection count column
nodes_df['Num_Connections'] = nodes_df['neighbors'].apply(len)

# Create a hover selection for edge highlighting on hover
hover_selection = alt.selection_point(
    fields=['Country'],
    on='mouseover',
    empty=False,
    clear='mouseout'
)

# Clickable nodes - attach the same country_selection for interactive clicks
clickable_nodes = alt.Chart(nodes_df).mark_circle().encode(
    x=alt.X('x:Q', scale=alt.Scale(domain=[-1, 1]), axis=None),
    y=alt.Y('y:Q', scale=alt.Scale(domain=[-1, 1]), axis=None),
    color=alt.condition(
        country_selection,
        alt.value('red'),
        alt.value('lightgrey')
    ),
    size=alt.condition(
        country_selection,
        alt.value(300),
        alt.value(150)
    ),
    tooltip=['Country:N', 'Num_Connections:Q']
).add_params(country_selection, hover_selection)

# All edges in light grey (base layer)
all_edges = alt.Chart(edges_df).mark_line(color="#FFFFFF", strokeWidth=1).encode(
    x=alt.X('source_x:Q', scale=alt.Scale(domain=[-1, 1]), axis=None),
    y=alt.Y('source_y:Q', scale=alt.Scale(domain=[-1, 1]), axis=None),
    x2='target_x:Q',
    y2='target_y:Q'
)

# Hover edges (medium grey)
hover_edges = alt.Chart(edges_df).mark_line(color='#808080', strokeWidth=2).encode(
    x=alt.X('source_x:Q', scale=alt.Scale(domain=[-1, 1]), axis=None),
    y=alt.Y('source_y:Q', scale=alt.Scale(domain=[-1, 1]), axis=None),
    x2='target_x:Q',
    y2='target_y:Q'
).transform_filter(
    hover_selection
)

# Selected edges (darkest)
selected_edges = alt.Chart(edges_df).mark_line(color='#333333', strokeWidth=3).encode(
    x=alt.X('source_x:Q', scale=alt.Scale(domain=[-1, 1]), axis=None),
    y=alt.Y('source_y:Q', scale=alt.Scale(domain=[-1, 1]), axis=None),
    x2='target_x:Q',
    y2='target_y:Q'
).transform_filter(
    country_selection
)

# Neighbor nodes (blue)
neighbor_nodes = alt.Chart(nodes_df).transform_filter(
    country_selection
).transform_flatten(
    ['neighbors'], as_=['neighbor_country']
).transform_lookup(
    lookup='neighbor_country',
    from_=alt.LookupData(nodes_df, 'Country', ['x', 'y', 'Country', 'Num_Connections'])
).mark_circle(size=150, color='steelblue').encode(
    x=alt.X('x:Q', scale=alt.Scale(domain=[-1, 1]), axis=None),
    y=alt.Y('y:Q', scale=alt.Scale(domain=[-1, 1]), axis=None),
    tooltip=['Country:N', 'Num_Connections:Q']
)

# Combine network layers (edges in order: all -> hover -> selected, then nodes)
network_chart = (all_edges + hover_edges + selected_edges + clickable_nodes + neighbor_nodes).properties(
    width=825,
    height=450,
    title=alt.TitleParams('Country Border Network (Click to select, Shift + Click for multiple)', fontSize=16)
)

# Combine network with connection counter vertically (network on top, counter below)
# Note: This visualization requires country_selection from the Map cell to be defined first
border_network_viz = alt.vconcat(network_chart)


# Bar Chart

In [6]:
# Build an edge-style DataFrame: one row per (country, neighbor) pair
rows = []
for idx, row in df.iterrows():
    country = row['Country']
    neighbors = row['Borders'] if isinstance(row['Borders'], list) else []
    for neigh in neighbors:
        rows.append({
            'Country': country,          # main country (the one you click)
            'neighbor_code': neigh       # its neighbor's 3-letter code
        })

neighbors_df = pd.DataFrame(rows)

# Selected country bar with red stroke border and interest rate color fill
selected_bar = alt.Chart(df).mark_bar(stroke='red', strokeWidth=3).encode(
    x=alt.X('Country:N', 
            title='Country', 
            sort='-y',
            axis=alt.Axis(titleFontSize=14, labelFontSize=12)),
    y=alt.Y('GDP_per_capita:Q', 
            title='GDP per Capita',
            axis=alt.Axis(titleFontSize=14, labelFontSize=12)),
    color=alt.Color('Interest Rate:Q', 
                    scale=alt.Scale(domain=[0, 20], clamp=True, scheme='viridis'), 
                    legend=alt.Legend(title='Interest Rate (%)', 
                                     orient='right', 
                                     values=[0, 5, 10, 15, 20],
                                     titleFontSize=14,
                                     labelFontSize=12)),
    tooltip=['Country:N', 'GDP_per_capita:Q', 'Interest Rate:Q']
).transform_filter(
    country_selection
)

# Neighbor countries bars with interest rate color fill, no stroke
neighbor_bar = alt.Chart(neighbors_df).transform_filter(
    country_selection        # uses 'Country' in neighbors_df
).transform_lookup(
    lookup='neighbor_code',
    from_=alt.LookupData(
        df,
        'Abbreviation_3',    # 3-letter code column in df
        ['Country', 'GDP_per_capita', 'Interest Rate']
    ),
    as_=['neighbor_name', 'neighbor_GDP_per_capita', 'neighbor_interest_rate']
).mark_bar().encode(
    x=alt.X('neighbor_name:N', 
            title='Country', 
            sort='-y',
            axis=alt.Axis(titleFontSize=14, labelFontSize=12)),
    y=alt.Y('neighbor_GDP_per_capita:Q', 
            title='GDP per Capita',
            axis=alt.Axis(titleFontSize=14, labelFontSize=12)),
    color=alt.Color('neighbor_interest_rate:Q', 
                    scale=alt.Scale(domain=[0, 20], clamp=True, scheme='viridis'), 
                    legend=alt.Legend(title='Interest Rate (%)', 
                                     orient='right',
                                     titleFontSize=14,
                                     labelFontSize=12)),
    tooltip=['neighbor_name:N', 'neighbor_GDP_per_capita:Q', 'neighbor_interest_rate:Q']
)

# Combine selected and neighbor bars
bar_chart = (selected_bar + neighbor_bar).properties(
    width=1650,
    height=350,
    title=alt.TitleParams('GDP per Capita (Selected Country + Neighbors)', fontSize=16)
)
bar_chart

alt.LayerChart(...)

# MCV

In [7]:
# Concatenate the interactive map and network horizontally, then add the bar chart below
interactive_combined_viz = alt.vconcat(
    alt.hconcat(population_density_map, border_network_viz),
    bar_chart
).resolve_scale(
    color='independent'
)
interactive_combined_viz

alt.VConcatChart(...)

In [8]:
# Save as standalone HTML
interactive_combined_viz.save('dashboard.html')